## Projecte Opengov 

The aim of this script is to scrappe the DOGC webpage in order to download all "Acord" document type and save them into a Mongo Database. 

## 1. Packages

In [351]:
#Packages
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import urllib, urllib2,cookielib, re
import pymongo
import os 
import numpy as np
import pandas as pd
import re
from time import sleep
from random import random

## 2. Database

In [352]:
#Create BD 
conn = pymongo.MongoClient()
db = conn['DB_opengov_castella']
DOGC_pdf= db['Persones']   
DOGC_pdf.remove()
DOGC_pdf.count()

/Users/xisca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


0

#### Helpers


In [298]:
## Deprecated 
def download_file(download_url):
    web_file = urllib.urlopen(download_url)
    local_file = open('some_file.pdf', 'w')
    local_file.write(web_file.read())
    web_file.close()
    local_file.close()

#if __name__ == 'main':
#   download_file('http://www.example.com/some_file.pdf')

## 3. Search

The first step is to run the browser using selenium and to get the pag ewith all results for a given name. 

### 3.1 List Documents

In [353]:
# Advanced search 
url = 'http://dogc.gencat.cat/ca/' #Url DOGC en catala 
browser = webdriver.Firefox() 
#browser = webdriver.Chrome()
browser.get(url)
sleep(random())

cerca_avan = browser.find_element_by_class_name('botonLink')
enllac= cerca_avan.find_element_by_class_name('enlace')
enllac.click()

In [4]:
# Search by name --- use this "cerca" to input your searched name
cerca = 'Meritxell Borras	'
text2 = '\"' + cerca + '\" '
print text2

"Meritxell Borras	" 


In [355]:
# Sent do doc 
cerca_paraula=browser.find_element_by_id('textWords')
cerca_paraula.clear()
cerca_paraula.send_keys(text2)

boto_cerca1=browser.find_element_by_id('cercaAv1')
boto_cerca1.click()

In [ ]:
delay = 5

In [356]:
#WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'mainContainer2')))
WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'mainContainer')))

quadre_resultats = browser.find_element_by_class_name('resultats')
#valor = int(re.findall(r'\d+', quadre_resultats.text)[0])
valor = int(''.join(re.findall(r'\d+', quadre_resultats.text)))
print valor

1232


In [357]:
if valor>200: 
    valor = 200 
print valor 
### Limit total of document to request 

200


#### 2.  Get documents, download anb save them into DB

In [342]:
#Download 
iter_number = 0
numero_document_insertats = 0

delay=5 
while True: 
    # Look for table of results 
    quadre_resultats = browser.find_element_by_class_name('resultats')
    llista_resultats = quadre_resultats.find_element_by_xpath('//*[@id="mainContainer"]/div/div/div[2]/div[2]/div[1]')
    #llista_resultats = quadre_resultats.find_element_by_xpath('//*[@id="mainContainer2"]/div/div/div[2]/div[2]/div[1]/div[2]/ul')
    llista_pdf = llista_resultats.find_elements_by_id('enllacFitxa0')
    llista_resultats_text = llista_resultats.text
    
    # Iterator
    contador = 1 
    for pdf_individual in llista_pdf:
        dic = {}
        dic['Cerca'] = cerca 
        dic['Area'] = llista_resultats_text.split('\n')[contador].split(',')[0].split(' ')[0]
        identificador=llista_resultats_text.split('\n')[contador].split(',')[0].replace('/','_').split(' ')[1:]
        identificador_nou = ''
        for paraules in identificador: 
            identificador_nou += paraules+' '
        dic['Identificador'] = identificador_nou
        dic['URL'] = pdf_individual.get_attribute('href')
        
        # Reading (open new driver) 
        name = cerca+'_'+llista_resultats_text.split('\n')[contador].split(',')[0].split(' ')[0]+'_'+''.join(identificador)
        if len(name)>=270: 
            name=name[0:100]
            
        url_pdf = pdf_individual.get_attribute('href')
        
        ## Using phantom browser 
        #browser_textifitxa = webdriver.Firefox() 
        browser_textifitxa = webdriver.PhantomJS('/Users/xisca/Desktop/Personal/Postgrau/Projecte/Scraping/phantomjs') 
        browser_textifitxa.get(url_pdf)
        
        # Check if page is working 
        no_funciona=0
        try: 
            browser_textifitxa.find_element_by_class_name('titol').text
            no_funciona=0
        except:
            no_funciona=1
        
        if no_funciona==0: 
            # If working, get title 
            titol_pdf = browser_textifitxa.find_element_by_class_name('titol').text
            dic['Titol']=titol_pdf
            # Get side data
            dades_document = browser_textifitxa.find_element_by_class_name('quadre')
            dades = dades_document.find_elements_by_tag_name('li')
            dic['Tipus Document']=dades[0].text.split('\n')[1]
            dic['Data del document']=dades[1].text.split('\n')[1]
            #dic['Número del document']=dades[2].text.split('\n')[1]
            #dic['Número del control']=dades[3].text.split('\n')[1]
            if len(dades[len(dades)-1].text.split('\n'))==1: 
                dic['Organisme Emisor']=dades[len(dades)-2].text.split('\n')[1] 
            else: 
                dic['Organisme Emisor']=dades[len(dades)-1].text.split('\n')[1] 
            # Get all text 
            fitxa_pdf = browser_textifitxa.find_element_by_class_name('fitxaFragment')
            dic['Text']=fitxa_pdf.text 
        
            # Update counter 
            contador = contador+2
            numero_document_insertats += 1
            DOGC_pdf.insert(dic)

            # Close browser 
            browser_textifitxa.close() 
        
            # Wait 
            sleep(random())
            
        if no_funciona==1:
            # If not working, update counter and close 
            contador = contador+2
            numero_document_insertats += 1
            #DOGC_pdf.insert(dic)

            # Close browser
            browser_textifitxa.close() 
            
            # Wait 
            sleep(random())
        
    # Next page 
    siguientes = browser.find_elements_by_class_name('enllacNegreta')
    for element in range(np.size(siguientes)): 
        if siguientes[element].get_attribute('title')== u'Seg\xfcent':
            siguiente=siguientes[element]    

    if numero_document_insertats >= valor-1:
        print 'All in!!!'
        break
    else:
        siguiente.click()
        url = browser.current_url
        browser.get(url) 
        
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'mainContainer2')))
        

        print 'Siguee mami'        
        #aw = browser.window_handles
        #browser.switch_to_window(aw[0])
        iter_number += 1   
        
        
        
browser.close()

/Users/xisca/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:69: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbda90>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbd610>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10b4f63d0>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x1150384d0>> ignored
Exception AttributeError: Attribute

Siguee mami
Siguee mami

AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115423910>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x108ba6f10>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115038410>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115423850>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115038350>> ignored



Siguee mami

AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10ace0f90>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x1150338d0>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10ace0550>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115ae4b50>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10ace0210>> ignored


...

AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x108ba6110>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115026510>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acb7e90>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbda10>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbdfd0>> ignored



Siguee mami

AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10ace0210>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbdfd0>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbdad0>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x1153f2450>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10acbd510>> ignored



All in!!!


AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x116e807d0>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x116e80450>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x116e80710>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x115ae4d90>> ignored
Exception AttributeError: AttributeError("'NoneType' object has no attribute 'close'",) in <bound method Service.__del__ of <selenium.webdriver.phantomjs.service.Service object at 0x10b51eed0>> ignored


In [343]:
print numero_document_insertats
print iter_number

96
7


#### 3.  Show summary of the database

In [344]:
data = pd.DataFrame(list(DOGC_pdf.find()))
print len(data) 
data.tail()

96


,Area,Cerca,Data del document,Identificador,Organisme Emisor,Text,Tipus Document,Titol,URL,_id
91,EDICTE,Meritxell Borras,05/10/2006,pel qual es fan públiques les candidatures pro...,Juntes Electorals,D'acord amb el que estableix l'article 47.5 de...,Edicte,EDICTE pel qual es fan públiques les candidatu...,http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d3395a9a603e44ad63abf3
92,EDICTE,Meritxell Borras,29/09/2006,pel qual es fan públiques les candidatures pre...,Juntes Electorals,D'acord amb el que estableix l'article 47.1 de...,Edicte,EDICTE pel qual es fan públiques les candidatu...,http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d339669a603e44ad63abf4
93,EDICTE,Meritxell Borras,20/10/2003,de 20 d'octubre de 2003,Juntes Electorals,"El senyor Jordi Seguí Puntas, president de la ...",Edicte,"EDICTE de 20 d'octubre de 2003, pel qual es fa...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d339729a603e44ad63abf5
94,EDICTE,Meritxell Borras,14/10/2003,de 14 d'octubre de 2003,Juntes Electorals,Candidatura núm. 1\nFormació política: Converg...,Edicte,"EDICTE de 14 d'octubre de 2003, pel qual es fa...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d3397a9a603e44ad63abf6
95,EDICTE,Meritxell Borras,20/09/1999,de 20 de setembre de 1999,Juntes Electorals,"Aquesta Junta Electoral Provincial, per Resolu...",Edicte,"EDICTE de 20 de setembre de 1999, pel qual es ...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d339859a603e44ad63abf7


In [346]:
filename = 'dogc_'+cerca.split(' ')[0]+'_'+cerca.split(' ')[1]+'.csv'
print filename


dogc_Meritxell_Borras.csv


In [347]:
data.to_csv(filename,encoding='utf-8')

In [348]:
print 'Numero filas: ', len(data.index)
print 'Numero columnas: ', len(data.columns)

Numero filas:  96
Numero columnas:  10


In [349]:
data_cast = pd.read_csv(filename,encoding='utf-8')
data_cast.head()

,Unnamed: 0,Area,Cerca,Data del document,Identificador,Organisme Emisor,Text,Tipus Document,Titol,URL,_id
0,0,DECRET,Meritxell Borras,30/09/2015,217_2015,Departament de la Presidència,La consellera de Governació i Relacions Instit...,Decret,"DECRET 217/2015, de 30 de setembre, d'encàrrec...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d335ec9a603e44ad63ab98
1,1,DECRET,Meritxell Borras,27/07/2015,184_2015,Departament de la Presidència,La consellera de Governació i Relacions Instit...,Decret,"DECRET 184/2015, de 27 de juliol, d'encàrrec d...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d335f29a603e44ad63ab99
2,2,DECRET,Meritxell Borras,22/06/2015,112_2015,Departament de la Presidència,D’acord amb els articles 12.1 d) i 17 de la Ll...,Decret,"DECRET 112/2015, de 22 de juny, pel qual es no...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d335f99a603e44ad63ab9a
3,3,LLEI,Meritxell Borras,21/07/2015,16_2015,Departament de la Presidència,El president de la Generalitat de Catalunya\n ...,Llei,"LLEI 16/2015, del 21 de juliol, de simplificac...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d336009a603e44ad63ab9b
4,4,DECRET,Meritxell Borras,19/01/2016,LLEI 1_2016,Departament de la Presidència,El president de la Generalitat de Catalunya.\n...,Decret llei,"DECRET LLEI 1/2016, de 19 de gener, d'aplicaci...",http://dogc.gencat.cat/ca/pdogc_canals_interns...,56d336069a603e44ad63ab9c
